In [1]:
#import necessary libraries
import pandas as pd
import nltk
import sys # needed this for certain print options during debugging
import numpy as np #lmao why did we not have this before
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/soumyankar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
#load the dataset
pd.set_option('display.max_colwidth', None)
df = pd.read_csv('data/final_dataset.csv')
df.head(2)

,review_id,title,tags,year,user_review,user_suggestion,url,Tokens,ProcessTokens,soup
0,1,Spooky's Jump Scare Mansion,"['Action', 'Adventure', 'Free to Play', 'Indie']",2016.0,"I'm scared and hearing creepy voices. So I'll pause for a moment and write a review while I wait for my heart beat to return to atleast somewhat calmer times. This game is adorable and creepy like my happy tree friends but with the graphics sceme of my childhood (but more bubble and 'clean'). Hello 1990's.What charactes there are (that isnot trying to kill me) were likable and a bit odd. I did do a few noob things though, such as:Oh look a class room full of ghosts from dead children, lets shine my flashlight on them and stand there staring at them..Or, hmm creepy music, I'll turn around and see if I can see what's chasing me.Never before in a game have I been this afraid of finding a locked door.",1,https://www.youtube.com/watch?v=48NzbFQebxg,"['I', ""'m"", 'scared', 'and', 'hearing', 'creepy', 'voices', '.', ' ', 'So', 'I', ""'ll"", 'pause', 'for', 'a', 'moment', 'and', 'write', 'a', 'review', 'while', 'I', 'wait', 'for', 'my', 'heart', 'beat', 'to', 'return', 'to', 'atleast', 'somewhat', 'calmer', 'times', '.', ' ', 'This', 'game', 'is', 'adorable', 'and', 'creepy', 'like', 'my', 'happy', 'tree', 'friends', 'but', 'with', 'the', 'graphics', 'sceme', 'of', 'my', 'childhood', '(', 'but', 'more', 'bubble', 'and', ""'"", 'clean', ""'"", ')', '.', ' ', 'Hello', ""1990's"", '.', 'What', 'charactes', 'there', 'are', '(', 'that', 'isnot', 'trying', 'to', 'kill', 'me', ')', 'were', 'likable', 'and', 'a', 'bit', 'odd', '.', ' ', 'I', 'did', 'do', 'a', 'few', 'noob', 'things', 'though', ',', 'such', 'as', ':', 'Oh', 'look', 'a', 'class', 'room', 'full', 'of', 'ghosts', 'from', 'dead', 'children', ',', 'lets', 'shine', 'my', 'flashlight', 'on', 'them', 'and', 'stand', 'there', 'staring', 'at', 'them', '..', 'Or', ',', 'hmm', 'creepy', 'music', ',', 'I', ""'ll"", 'turn', 'around', 'and', 'see', 'if', 'I', 'can', 'see', 'what', ""'s"", 'chasing', 'me', '.', 'Never', 'before', 'in', 'a', 'game', 'have', 'I', 'been', 'this', 'afraid', 'of', 'finding', 'a', 'locked', 'door', '.']","['hearing', 'creepy', 'pause', 'moment', 'write', 'review', 'wait', 'heart', 'beat', 'return', 'somewhat', 'calmer', 'time', 'game', 'adorable', 'creepy', 'like', 'happy', 'tree', 'graphic', 'childhood', 'bubble', 'clean', 'hello', 'trying', 'kill', 'likable', 'bit', 'odd', 'oh', 'look', 'class', 'room', 'dead', 'shine', 'flashlight', 'stand', 'staring', 'creepy', 'music', 'turn', 'chasing', 'game', 'afraid', 'finding', 'locked', 'door']",hearing creepy pause moment write review wait heart beat return somewhat calmer time game adorable creepy like happy tree graphic childhood bubble clean hello trying kill likable bit odd oh look class room dead shine flashlight stand staring creepy music turn chasing game afraid finding locked door action adventure free to play indie
1,2,Spooky's Jump Scare Mansion,"['Action', 'Adventure', 'Free to Play', 'Indie']",2016.0,"Best game, more better than Sam Pepper's YouTube account. 10/10What you'll need to play:A computerSome extra pants.Pros:Scary as hell.Fun.Adventure.Spooky.Did I forgot to mention that its scary as hell?You'll get more pants/briefs in your wardrobe.Time consuming if you're bored.Cons:Buying pants/briefs. You haven't downloaded it yet.",1,https://www.youtube.com/watch?v=48NzbFQebxg,"['Best', 'game', ',', 'more', 'better', 'than', 'Sam', 'Pepper', ""'s"", 'YouTube', 'account', '.', '10/10What', 'you', ""'ll"", 'need', 'to', 'play', ':', 'A', 'computerSome', 'extra', 'pants', '.', 'Pros', ':', 'Scary', 'as', 'hell', '.', 'Fun', '.', 'Adventure', '.', 'Spooky', '.', 'Did', 'I', 'forgot', 'to', 'mention', 'that', 'its', 'scary', 'as', ""hell?You'll"", 'get', 'more', 'pants', '/', 'briefs', 'in', 'your', 'wardrobe', '.', 'Time', 'consuming', 'if', 'you', ""'re"", 'bored', '.', 'Cons', ':', 'Buying', 'pants', '/', 'briefs', '.', 'You', 'have', "

In [5]:
#Converting the datatype of processtokens fromm list to string, so that our vectorization4 works right.
df['ProcessTokens'] = df['ProcessTokens'].astype("string")

In [6]:
def getSearchTerms():
    genres = ['Action', 'Adventure', 'Casual', 'Free to Play', 'Massively Multiplayer', 'RPG']
    review1 = ["I love plains of eidolon, let's remove every mission in the game and put their rewards in the plains so we can play max range nova every game."]
    review2 = []
    review3 = []
    searchTerms = genres + review1 + review2 + review3
    
    searchTerms = " ".join(map(str, searchTerms))
    searchTerms = searchTerms.lower()
    return searchTerms

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from itertools import chain

def make_recommendation(df=df):
    new_row = df.iloc[-1,:].copy() #creating a copy of the last row of the 
    #dataset, which we will use to input the user's input

    #grabbing the new wordsoup from the user
    searchTerms = getSearchTerms()
    new_row['soup'] = searchTerms
    new_row['title'] = 'UserInput'

    #adding the new row to the dataset
    df = df.append(new_row)
#     df.iloc[-1] = searchTerms #adding the input to our new row

    #Vectorizing the entire matrix as described above!
    count = CountVectorizer(stop_words='english')
    count_matrix = count.fit_transform(df['soup'])

    #running pairwise cosine similarity 
    cosine_sim2 = cosine_similarity(count_matrix, count_matrix) #getting a similarity matrix

    #sorting cosine similarities by highest to lowest
    sim_scores = list(enumerate(cosine_sim2[-1,:]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    #matching the similarities to the movie titles and ids
    #   ranked_titles = []
    #   for i in range(1, 11):
    #     indx = sim_scores[i][0]
    #     ranked_titles.append([df['title'].iloc[indx], df['soup'].iloc[indx]])
    i = 10
    inc = 1 # we start from the 1-th row because, the 0th row is the input itself. dont want that.
    
    ranked_titles = []
    while i>=0:
        indx = sim_scores[inc][0]
        inc = inc + 1
        current_title = df['title'].iloc[indx]
        if(current_title == 'UserInput'):
            continue
        if(current_title not in chain(*ranked_titles)):
            ranked_titles.append([df['title'].iloc[indx], sim_scores[inc-1][1]])
            i = i - 1

    return ranked_titles

In [8]:
import time
start_time = time.time()
#let's try our recommendation function now
user_recs = make_recommendation()
display(user_recs)
print("--- %s seconds ---" % (time.time() - start_time))

/var/folders/r3/y82b8vjs51z3zz5mznqcwwbc0000gn/T/ipykernel_12224/2116405780.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row)


[['School of Dragons', 0.6267831705280087],
 ['Elsword', 0.6213697660012],
 ['Creativerse', 0.6172133998483675],
 ['Realm of the Mad God', 0.6123724356957945],
 ['Neverwinter', 0.607142857142857],
 ['Path of Exile', 0.6047431568147635],
 ['Trove', 0.6043672230190351],
 ['AdventureQuest 3D', 0.602763781115103],
 ['Infestation: The New Z', 0.5958540230683373],
 ['World of Tanks Blitz', 0.5910828046793255],
 ['Robocraft', 0.5819143739626463]]

--- 31.393511056900024 seconds ---
